<a href="https://colab.research.google.com/github/m2gu3el/Engineering-QnA-chatbot/blob/main/Engineering_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Libraries needed for NLP
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# Libraries needed for Tensorflow processing
!pip install tflearn
import numpy as np
import tensorflow as tf
!pip uninstall tflearn
!pip install git+https://github.com/MihaMarkic/tflearn.git@fix/is_sequence_missing
import tflearn
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Found existing installation: tflearn 0.5.0
Uninstalling tflearn-0.5.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/tflearn-0.5.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/tflearn/*
Proceed (Y/n)? y
  Successfully uninstalled tflearn-0.5.0
  Cloning https://github.com/MihaMarkic/tflearn.git (to revision fix/is_sequence_missing) to /tmp/pip-req-build-sdq_kpq9
  Running command git clone --filter=blob:none --quiet https://github.com/MihaMarkic/tflearn.git /tmp/pip-req-build-sdq_kpq9
  Running command git checkout -b fix/is_sequence_missing --track origin/fix/is_sequence_missing
  Switched to a new branch 'fix/is_sequence_missing'
  Branch 'fix/is_sequence_missing' set up to track remote branch 'fix/is_sequence_missing' from 'origin'.
  Resolved https://github.com/MihaMarkic/tflearn.git to commit 6472b8588e758ff4a33a2764d4ee638bbd0e42f0
  Preparing metadata (setup.py) ... done
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=130659 sha

In [ ]:
from google.colab import files
files.upload()

Saving intents.json to intents.json


{'intents.json': b'{\r\n    "intents": [\r\n        {\r\n            "tag": "greeting",\r\n            "patterns": ["Hi", "Hello", "Hey", "Greetings"],\r\n            "responses": ["Hello! How can I assist you with your college admission?", "Hi there! How can I help you today?", "Welcome! How can I support you in your college journey?"],\r\n            "context_set": ""\r\n        },\r\n        {\r\n            "tag": "goodbye",\r\n            "patterns": ["Goodbye", "Farewell", "See you later", "Bye"],\r\n            "responses": ["Goodbye! If you have more questions, feel free to ask.", "Farewell! If you need further assistance, don\'t hesitate to reach out.", "See you later! Best of luck with your college endeavors."],\r\n            "context_set": ""\r\n        },\r\n        {\r\n            "tag": "thanks",\r\n            "patterns": ["Thanks", "Thank you", "Appreciate it"],\r\n            "responses": ["You\'re welcome! If you have more questions, feel free to ask.", "Happy to he

In [ ]:
# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [ ]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'Hello', 'Hey', 'Greetings'],
   'responses': ['Hello! How can I assist you with your college admission?',
    'Hi there! How can I help you today?',
    'Welcome! How can I support you in your college journey?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['Goodbye', 'Farewell', 'See you later', 'Bye'],
   'responses': ['Goodbye! If you have more questions, feel free to ask.',
    "Farewell! If you need further assistance, don't hesitate to reach out.",
    'See you later! Best of luck with your college endeavors.'],
   'context_set': ''},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', 'Appreciate it'],
   'responses': ["You're welcome! If you have more questions, feel free to ask.",
    "Happy to help! If there's anything else, just let me know.",
    'Glad I could assist you! If you need further information, feel free to inquire.']},
  {'tag': 'admission_requirements',
   'patterns': ['What are the ad

In [ ]:
words = []
classes = []
documents = []
ignore = ['?']
# loop through each sentence in the intent's patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each and every word in the sentence
        w = nltk.word_tokenize(pattern)
        # add word to the words list
        words.extend(w)
        # add word(s) to documents
        documents.append((w, intent['tag']))
        # add tags to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [ ]:
# Perform stemming and lower each word as well as remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

# remove duplicate classes
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

26 documents
8 classes ['admission_requirements', 'application_process', 'contact_admissions', 'deadlines', 'financial_aid', 'goodbye', 'greeting', 'thanks']
49 unique stemmed words ['about', 'admit', 'aid', 'an', 'apply', 'apprecy', 'ar', 'avail', 'bye', 'can', 'contact', 'counsel', 'criter', 'deadlin', 'detail', 'do', 'farewel', 'fin', 'for', 'get', 'goodby', 'greet', 'hello', 'hey', 'hi', 'how', 'i', 'in', 'involv', 'is', 'it', 'lat', 'me', 'nee', 'process', 'requir', 'scholarships', 'see', 'speak', 'step', 'support', 'tel', 'thank', 'the', 'timelin', 'to', 'what', 'when', 'you']


In [ ]:
# create training data
training = []
output = []
# create an empty array for output
output_empty = [0] * len(classes)

# create training set, bag of words for each sentence
for doc in documents:
    # initialize bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stemming each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is '1' for current tag and '0' for rest of other tags
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffling features and turning it into np.array
random.shuffle(training)
training = np.array(training)

# creating training lists
train_x = list(training[:,0])
train_y = list(training[:,1])

<ipython-input-9-0222230fb184>:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [ ]:
# resetting underlying graph data
#tf.reset_default_graph()

# Building neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Defining model and setting up tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

# Start training
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 3999  | total loss: 0.00394 | time: 0.014s
| Adam | epoch: 1000 | loss: 0.00394 - acc: 1.0000 -- iter: 24/26
Training Step: 4000  | total loss: 0.00402 | time: 0.019s
| Adam | epoch: 1000 | loss: 0.00402 - acc: 1.0000 -- iter: 26/26
--


In [ ]:
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

In [ ]:
# restoring all the data structures
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

In [ ]:
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [ ]:
# load the saved model
model.load('./model.tflearn')

In [ ]:
def clean_up_sentence(sentence):
    # tokenizing the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stemming each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# returning bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenizing the pattern
    sentence_words = clean_up_sentence(sentence)
    # generating bag of words
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [ ]:
ERROR_THRESHOLD = 0.30
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # a random response from the intent
                    return print(random.choice(i['responses']))

            results.pop(0)

In [ ]:
classify('Hello')

[('greeting', 0.9969271)]

In [ ]:
response('Hi')

Hi there! How can I help you today?


In [ ]:
response('How are you?')

Application deadlines vary by program. You can find detailed information on our website or specify the program you're interested in for more accurate details.


In [ ]:
#Some of other context free responses.
response('How the day')

We offer various financial aid options, including scholarships, grants, and loans. You can find more information on our website or specify the type of financial aid you're interested in for personalized details.


In [ ]:
response('entrance')

Welcome! How can I support you in your college journey?


In [ ]:
response('admission')

Application deadlines vary by program. You can find detailed information on our website or specify the program you're interested in for more accurate details.


In [ ]:
response('Bye')

Goodbye! If you have more questions, feel free to ask.


In [ ]:
#Adding some context to the conversation i.e. Contexualization for altering question and intents etc.
# create a data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return print(random.choice(i['responses']))

            results.pop(0)

In [ ]:
response('how to apply?')

The application process involves several steps, including submitting an online application, providing transcripts, and possibly letters of recommendation. Would you like more detailed information on a specific step or program?


In [ ]:
response('transcripts')

Glad I could assist you! If you need further information, feel free to inquire.
